# A. All Needed Modules

In [297]:
# !pip install torch
# !pip install pdfminer
# !pip install pytesseract
# !pip install tensorflow
# !pip install pdf2image
# !pip install transformers
# !pip install spacy
# !spacy download en_core_web_lg
# !pip install opencv-python
# !pip install seqeval
# !pip install evaluate
# !pip install huggingface_hub

# B. Account Login

In [2]:
# from huggingface_hub import notebook_login
# from huggingface_hub import upload_file
# from huggingface_hub import delete_file
# from huggingface_hub import Repository
# notebook_login()

# C. Import Modules

In [299]:
#PDF Miner (PDF Scraping)
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import HTMLConverter, TextConverter, XMLConverter
from io import StringIO
from pdfminer.pdfpage import PDFPage

#OCR
from pdf2image import convert_from_path
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r'Tesseract-OCR\tesseract.exe'


#NLP Preprocessing
import re
import regex


#NLP Processing TensorFLow
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

#NLP Processing Spacy
import spacy
spc = spacy.load('en_core_web_lg')

#Model Training and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import torch

#Basic
import numpy as np
import pandas as pd
import pickle
import glob
import os
# import docker
import string
import matplotlib.pyplot as plt

pc = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~'

from datasets.dataset_dict import DatasetDict
from datasets import Dataset

from transformers import DataCollatorForTokenClassification

# D. TRANSFER LEARNING of Fine-Tuned Model: GeoBERT_NER

In [300]:
# Import Tokenizer architecture from GeoBERT
from transformers import AutoTokenizer

model_checkpoint = "botryan96/GeoBERT_NER"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## D.1 All Functions for Self Supervised - Rule Based Auto Labeling

In [301]:
def get_training_corpus():
    for start_idx in df.sent.values.tolist():
        yield start_idx
        
#Ready for NLP NER V8.0 (BIO Encoding V2)
def bio_encoding(sent):
    #Tokenization

    #Preparation
    r = []
    list_word = []
    a4_token = []
    b4_token = sent.split(' ')
    for b in b4_token:
        a4_token.append(tokenizer(b).tokens()[1:-1])
        r.append(tokenizer(b).tokens()[1:-1])

    #Check entities in the sentence.
    for j,k in enumerate(list(dict_entities.values())):
        pat = r",*(\b(?:{})(?:e*s*))\b".format("|".join(k))
        se = sent.lower()
        sear = re.findall(pat,se)


        #Adjust Tuple into List format from regex.findall result 
        try:
            if type(sear[0]) == tuple:
                c = []
                for i,m in enumerate(sear):
                    c.append(list(filter(None, [d for d in m])))
                c = sum(c,[])
                sear=list(c)
            else:
                pass
        except:
            pass

        #Grab all entities for BIO Encoding
        if bool(re.search(pat,se)) == True:
            for l in adjust_order(sear):
                list_word.append(l)
                try:
                    length_l = len(l.split(' '))

                    #Check the position of entities and change it
                    ind_pos = regex.sub(r'{}'.format(l),'O',se).split(' ').index('O')
                    for i in range(length_l):
                        try:
                            num = b4_token.index(l.split(' ')[0], ind_pos, ind_pos+1)
                        except:
                            pass
                        b4_token[ind_pos + i] = 'Z'

                    se = ' '.join(b4_token)
                    
                    #Grab entities with more than one words
                    if length_l > 1:
                        k=0
                        for d,h in enumerate(a4_token[num]): 
                            if k == 0:
                                a4_token[num][d] = 'B-' + list(dict_entities.keys())[j]
                                k = k + 1
                            elif k != 0:
                                a4_token[num][d] = 'I-' + list(dict_entities.keys())[j]
                            else:
                                pass

                        h = 0
                        for k in l.split(' ')[1:]:
                            h = h + 1
                            for s,a in enumerate(a4_token[num + h]):
                                if a4_token[num + h][s]:
                                    a4_token[num + h][s] = 'I-' + list(dict_entities.keys())[j]
                                else:
                                    pass

                    #Grab entities with one word
                    elif length_l == 1:
                        k=0
                        for d,h in enumerate(a4_token[ind_pos]): 
                            if k == 0:
                                a4_token[num][d] = 'B-' + list(dict_entities.keys())[j]
                                k = k + 1
                            elif k != 0:
                                a4_token[num][d] = 'I-' + list(dict_entities.keys())[j]
                            else:
                                pass
                    
                    #Normalize the Sentence
                    sent = " ".join(sum(a4_token,[]))        
                except:
                    pass
    
    pad = [None] 
    final = pad + sum(a4_token,[]) + pad
    
    list_map = list(ent_map.keys())
    for i,j in enumerate(final):
        if j in list(list_map) or j == None:
            pass
        else:
            final[i] = 'O'
            
    return final, list(zip(r,a4_token)), list_word


def ner_encoded(input_ids):
    return [ent_map[str(i)] for i in input_ids]

def aturi(c):
    d = []
    d_num = []
    for i,j in enumerate(c):
        if type(j) == str:
            d.append(j)
            d_num.append(i)
        elif type(j) == list:
            for k in j:
                d.append(k)
                d_num.append(i)
        else:
            pass
    return(d, d_num)

def bio_encoding_predefined(m_token, m_code):
    #Define the predefined Tokens Result
    b4_token = list(m_token)

    #Define the current Tokens Result
    prep = tokenizer(' '.join(m_token))

    a4_token = []
    for i in set(prep.word_ids()[1:-1]):
        start, end = prep.word_to_chars(i)
        a4_token.append(' '.join(b4_token)[start:end].lower())

    beda = list(set([i for i in b4_token if i.lower() not in a4_token]))

    #Solution for different Tokenization results
    #If there is different in Tokenization result
    if len(beda) != 0:
        dd_array = np.array(b4_token)
        for i in beda:
            ind = np.where(dd_array==i)[0]
            for k in ind:
                b4_token[k] = tokenizer(b4_token[k]).tokens()[1:-1]

        #Adjusment for Pre-defined Tokenization
        dic_1={}
        for i,j in enumerate(m_code):
            dic_1[i] = j

        aturi_op = aturi(b4_token)

        input_ids = [dic_1[j] for j in aturi_op[1]]

        prep_at = np.array(aturi_op[1])

        for i in list(set(aturi_op[1])):
            res = np.where(prep_at == i)
            if len(res[0]) > 1:
                for i in res[0][1:]:
                    input_ids[i] = re.sub(r'B-','I-',input_ids[i])
            else:
                pass

        #Adjusment for Current Tokenization
        dic_2={}
        for i,j in enumerate(input_ids):
            dic_2[i] = j

        input_ids = [dic_2[j] if j!= None else None for j in prep.word_ids()]

        prep_at = np.array(prep.word_ids())

        for i in list(set(prep.word_ids()[1:-1])):
            res = np.where(prep_at == i)
            if len(res[0]) > 1:
                for i in res[0][1:]:
                    input_ids[i] = re.sub(r'B-','I-',input_ids[i])
            else:
                pass

    #If there is no different in Tokenization result
    else:
        #Adjusment for Current Tokenization
        dic_1={}
        for i,j in enumerate(m_code):
            dic_1[i] = j

        input_ids = [dic_1[j] if j!= None else None for j in prep.word_ids()]

        prep_at = np.array(prep.word_ids())

        for i in list(set(prep.word_ids()[1:-1])):
            res = np.where(prep_at == i)
            if len(res[0]) > 1:
                for i in res[0][1:]:
                    input_ids[i] = re.sub(r'B-','I-',input_ids[i])
            else:
                pass

    print('New Tokenization Result:')
    print('-------')
    print(list(zip(prep.tokens(),input_ids)))
    
    return prep, input_ids

def adjustment(data,sent):
    if len(set(data)) == 2:
        data = tokenizer(sent).word_ids()
    else:
        pass
    return data

def adjust_order(r):
    content_com = []
    max_space = max([len(re.findall(r"\s",i)) for i in r])

    for i in range(0,max_space+1):
        locals()['space_'+str(i)] = []

    for i in r:
        s = re.findall(r"\s",i)
        locals()['space_'+str(len(s))].append(i)

    for i in range(max_space,-1,-1):
        content_com = content_com + locals()['space_'+str(i)]
    
    return content_com

def adjust_first(sent):
    #Sentence Treatment b4 tokenization
    pattern1 = regex.compile(r'[]!"$%&\'()’–*´·+,./:;=#@?[\\^_`{|}~-]+') #Spacing
    pattern2 = regex.compile(r'[^\w&\'\)\(,-./´·–’]') #only used punctuation
    pattern3 = regex.compile(r'(\s)(?=\s+)') #Over Spacing
    
    sent_clear = regex.sub(pattern1, r' \g<0> ', sent.lower()).strip()
    sent_clear = regex.sub(pattern2,' ',sent_clear)
    sent_clear = regex.sub(pattern3,'',sent_clear)
    
    return sent_clear

# E. Read & Prepare the Text Data for Geoscience & Non-Geoscience

In [303]:
#General Corpus
df_6 = pd.read_csv('OTHER DATA/Fake - True News/True.csv/True.csv', error_bad_lines=False)

#Text Conditioning
compiler = regex.compile('^[\w\W]+\s-\s')
df_6.text = df_6.text.apply(lambda x : regex.sub(compiler,'',x))
df_6.subject = df_6.text.apply(lambda x : 0)

#Data Subset
df_ready_general = df_6[['title','subject']]
df_ready_general.rename(columns = {'subject':'label'}, inplace = True)
df_ready_general.rename(columns = {'title':'sentence'}, inplace = True)
df_ready_general.sentence = df_ready_general.sentence.apply(lambda x : x + ".")
df_ready_general.sentence = df_ready_general.sentence.apply(lambda x : adjust_first(x))

df_ready_general

C:\Users\Admin\AppData\Local\Temp\ipykernel_12100\2454391182.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_6 = pd.read_csv('OTHER DATA/Fake - True News/True.csv/True.csv', error_bad_lines=False)
C:\Users\Admin\AppData\Local\Temp\ipykernel_12100\2454391182.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ready_general.rename(columns = {'subject':'label'}, inplace = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_12100\2454391182.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ready_general.rename(colu

,sentence,label
0,"as u . s . budget fight looms , republicans fl...",0
1,u . s . military to accept transgender recruit...,0
2,senior u . s . republican senator ' let mr . m...,0
3,fbi russia probe helped by australian diplomat...,0
4,trump wants postal service to charge ' much mo...,0
...,...,...
21412,' fully committed ' nato backs new u . s . app...,0
21413,lexisnexis withdrew two products from chinese ...,0
21414,minsk cultural hub becomes haven from authorit...,0
21415,vatican upbeat on possibility of pope francis ...,0


In [305]:
#Geoscience Corpus
df_5 = pd.read_csv('partition_full/part_8_full_auto.csv')
df_5.drop('Unnamed: 0', inplace = True, axis=1)
df_geo = df_5.copy()[0:21417]
df_geo.rename(columns = {'sent':'sentence'}, inplace = True)
df_geo['label'] = df_geo.sentence.apply(lambda x : 1)
df_geo

,sentence,label
0,hydraulic heads above the stope are high and t...,1
1,the total flux into the stope for the base cas...,1
2,results for each case were compared to this va...,1
3,6 simulated hydraulic heads and velocity vecto...,1
4,7 a flux into the stope vs . hydraulic conduct...,1
...,...,...
21412,ecosystems of this kind have not yet undergone...,1
21413,representing an early stage of ecological succ...,1
21414,1 location of investigated area full size imag...,1
21415,it was also assumed that determining the lake ...,1


In [307]:
#Combine & Shuffle the data position
df_ready = pd.concat([df_ready_general, df_geo ]).sample(frac=1).reset_index(drop=True)
df_ready

,sentence,label
0,"hence , the groundwater chemistry was determin...",1
1,"the dry matter , which represents all of the o...",1
2,fracture criterion and sifs for a mixed mode c...,1
3,"1a , b ), where gcl - s had a relatively highe...",1
4,dissolution of aluminosilicate minerals induce...,1
...,...,...
42829,"hurricane maria now category 4 , puerto rico l...",0
42830,"by using boxes of size l , the minimum numbers...",1
42831,uk minister offers parliament new vote on any ...,0
42832,"therefore , an interval number is used to refl...",1


In [ ]:
# #Read the already prepared data
# df_ready = pd.read_csv('tes_ready.csv', error_bad_lines=False)
# df_ready.drop('Unnamed: 0', inplace = True, axis=1)

In [308]:
# Splitting Data (Train, Test, and Validation) in Dataset Dictionary format
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

train_dataset, validation_dataset = train_test_split(df_ready[["sentence", "label"]], test_size=0.30, random_state=42)
validation_dataset, test_dataset = train_test_split(validation_dataset, test_size=0.2, random_state=42)

train_dataset = Dataset.from_dict(train_dataset)
test_dataset = Dataset.from_dict(test_dataset)
validation_dataset =  Dataset.from_dict(validation_dataset)
my_dataset_dict = DatasetDict({"train":train_dataset,"test":test_dataset, "validation": validation_dataset})

# F. Prepare the Pre-Processed 2 dataset

In [309]:
#Function for preprocessing 2
def preprocess_function(examples):
    return tokenizer(examples['sentence'], truncation=True)

In [310]:
# Test Function for preprocessing 2
preprocess_function(my_dataset_dict["train"][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[2, 149, 285, 683, 158, 1, 2257, 12, 159, 325, 11, 3], [2, 1595, 3309, 1179, 6, 42, 6, 14452, 1560, 6142, 99, 95, 6, 5280, 6628, 85, 219, 1552, 11544, 4255, 110, 14533, 5584, 161, 11, 3], [2, 336, 205, 6, 42, 736, 16786, 10652, 95, 249, 8155, 7177, 1831, 373, 175, 12017, 2143, 460, 252, 11, 3], [2, 309, 505, 1797, 217, 149, 532, 1454, 188, 1173, 1382, 198, 1224, 2061, 11, 3], [2, 822, 674, 858, 608, 14454, 176, 6269, 95, 234, 660, 1447, 176, 1795, 7488, 1379, 96, 969, 1351, 20155, 18436, 227, 11, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1

In [311]:
#Build Dataset of Preprocessed 2 data
pre_tokenizer_columns = set(my_dataset_dict["train"].features)
encoded_dataset = my_dataset_dict.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

Columns added by tokenizer: ['token_type_ids', 'attention_mask', 'input_ids']


In [312]:
encoded_dataset["train"].features["label"]

Value(dtype='int64', id=None)

# G. GeoBERT_Analyzer Data Training

In [313]:
#Define the source of model (GeoBERT) for Transfer Learning by loading the previous Weights

#Define the use case. Here, to do labeling of text sentence, TFAutoModelforSequenceClassification is used
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf

#Define Total Labels (2)
num_labels = 2
id2label = { 0:"general", 1:"geoscience"}
label2id = {val: key for key, val in id2label.items()}

#Define all Model Parameters
model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

Some layers from the model checkpoint at botryan96/GeoBERT were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at botryan96/GeoBERT and are newly initialized: ['bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of TFBertForSequenceClassification were

In [314]:
#Prepare the data Training and Validation for the GeoBERT_Analyzer training
tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset["train"],
    shuffle=True,
    batch_size=5,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    encoded_dataset["validation"],
    shuffle=False,
    batch_size=5,
    tokenizer=tokenizer,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [316]:
#Define Training Optimization Fine-Tuning Hyperparameter
from transformers import create_optimizer
batch_size =5
num_epochs = 10

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [317]:
#Conduct the Training
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs
)

Epoch 1/10
5996/5996 [==============================] - 5690s 943ms/step - loss: 0.0254 - val_loss: 0.0067
Epoch 2/10
5996/5996 [==============================] - 5671s 946ms/step - loss: 0.0057 - val_loss: 0.0058
Epoch 3/10
5996/5996 [==============================] - 5637s 940ms/step - loss: 0.0034 - val_loss: 0.0095
Epoch 4/10
5996/5996 [==============================] - 5638s 940ms/step - loss: 0.0017 - val_loss: 0.0098
Epoch 5/10
5996/5996 [==============================] - 5655s 943ms/step - loss: 4.7795e-04 - val_loss: 0.0185
Epoch 6/10
5996/5996 [==============================] - 5643s 941ms/step - loss: 6.9776e-04 - val_loss: 0.0139
Epoch 7/10
5996/5996 [==============================] - 5637s 940ms/step - loss: 1.5166e-05 - val_loss: 0.0176
Epoch 8/10
5996/5996 [==============================] - 5649s 942ms/step - loss: 1.2517e-06 - val_loss: 0.0209
Epoch 9/10
5996/5996 [==============================] - 5653s 943ms/step - loss: 1.4787e-04 - val_loss: 0.0196
Epoch 10/10
5996/

# G. Training Evaluation

In [318]:
#Evaluation
import evaluate
metric = evaluate.load("seqeval")
import numpy as np
label_names = list(label2id.keys())

all_predictions = []
all_labels = []
k = 1
for batch in tf_validation_dataset:
    a = batch
    logits = model.predict(batch)["logits"]
    labels = batch[1]
    predictions = np.argmax(logits, axis=-1)
    for i,j in zip(predictions.tolist(), labels.numpy().tolist()):
        all_predictions.append(label_names[i])
        all_labels.append(label_names[j])
metric.compute(predictions=[all_predictions], references=[all_labels])

1/1 [==============================] - 0s 266ms/step


1/1 [==============================] - 1s 531ms/step


1/1 [==============================] - 0s 234ms/step


1/1 [==============================] - 0s 391ms/step


1/1 [==============================] - 0s 328ms/step


1/1 [==============================] - 1s 719ms/step


1/1 [==============================] - 0s 281ms/step


C:\Users\Admin\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: general seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\Admin\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: geoscience seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eneral': {'precision': 0.9942660550458715,
  'recall': 0.9935064935064936,
  'f1': 0.9938861291555217,
  'number': 2618},
 'eoscience': {'precision': 0.994263862332696,
  'recall': 0.9935040122277417,
  'f1': 0.9938837920489296,
  'number': 2617},
 'overall_precision': 0.9942649588988721,
 'overall_recall': 0.993505253104107,
 'overall_f1': 0.9938849608255304,
 'overall_accuracy': 0.998443579766537}

# H. Model GeoBERT_Analyzer Deployment

In [146]:
sentences = ['the average iron and sulfate concentrations were calculated to be 19 . 6 5 . 2 and 426 182 mg / l , respectively .',
            'She first gained media attention as a friend and stylist of Paris Hilton']

tokenized = tokenizer(sentences, return_tensors="np", padding="longest")

outputs = model(tokenized).logits

classifications = np.argmax(outputs, axis=1)
print(classifications)

classifications = [model.config.id2label[output] for output in classifications]
print(classifications)

[1 0]
['geoscience', 'general']


In [1]:
# repo = Repository("GeoBERT_analyzer", clone_from="botryan96/GeoBERT_analyzer")
# model.save_pretrained(repo.local_dir)
# tokenizer.save_pretrained(repo.local_dir)
# repo.git_pull()
# repo.git_add()
# repo.git_commit("GeoBERT Analyzer")
# repo.git_push()

In [149]:
outputs

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-5.282356 ,  5.5434365],
       [ 1.895186 , -2.0161612]], dtype=float32)>